In [1]:
import numpy as np
import scipy as sp
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt

In [2]:
from tensorflow.keras import optimizers
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import metrics
from keras.utils import to_categorical

Using TensorFlow backend.


In [3]:
import time
import os
import sys
import pickle
from minibatch import *

In [4]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [5]:
db_dir = "/home/sweet/2-coursework/725bmes/eeg_deeplearning/data_extraction/src"
ME_db_fname = "prelim_ME_db.pickle"
# MI_db_fname = "prelim_MI_db.pickle"

ME_db = {}
# MI_db = {}


t1 = time.time()
with open(db_dir + "/" + ME_db_fname, "rb") as f:
    ME_db = pickle.load(f)
print("Loaded ME database in %f s" % (time.time()-t1))

np.random.seed(0)
seq_perms = create_rand_seq_permutations(ME_db)
np.random.seed(0)
batch_gen = generate_batch(ME_db, seq_perms, 4)
output_seq, labels = next(batch_gen)
print(output_seq[0][0].shape)

Loaded ME database in 0.900293 s
(64,)


In [6]:
# def build_conv2d(inputs, filters, kernel_size, strides=(1, 1), padding='same', activation=None, name=None, reuse=None):
#     return tf.layers.conv2d(inputs=inputs, filters=filters, kernel_size=kernel_size, strides=strides, padding=padding, activation=activation,
#                 kernel_initializer=tf.truncated_normal_initializer(stddev=0.1), bias_initializer=tf.constant_initializer(0.1), name=name, reuse=reuse)

In [7]:
# def create_CNN_stack(nstack):
#     input_layer = layers.Input(shape=(10,11,1))
#     conv2d_L1 = layers.Conv2D(32, 3, activation="relu", padding="same", 
#                               name="CNN_layer1"+"_"+str(nstack))(input_layer)
#     conv2d_L2 = layers.Conv2D(64, 3, activation="relu", padding="same", 
#                               name="CNN_layer2"+"_"+str(nstack))(conv2d_L1)
#     conv2d_L3 = layers.Conv2D(128, 3, activation="relu", padding="same", 
#                               name="CNN_layer3"+"_"+str(nstack))(conv2d_L2)
#     flat = layers.Flatten()(conv2d_L3)
#     dense = layers.Dense(1024, activation="relu", name="CNN_FC"+"_"+str(nstack))(flat)
#     output_layer = layers.Dropout(dropout_rate, name="CNN_drop"+"_"+str(nstack))(dense)

#     return input_layer, output_layer

# def create_model_cascade():
#     input_layers = []
#     cnn_outputs = []

#     for i in range(S):
#         inp, out = create_CNN_stack(i)
#         input_layers.append(inp)
#         cnn_outputs.append(out)

#     cnn_stack_model = models.Model(inputs=input_layers, outputs=cnn_outputs)

#     concat_CNN_ouputs = layers.concatenate(cnn_stack_model.outputs)
#     concat_CNN_ouputs = layers.Reshape((h_states_lstm, concat_CNN_ouputs.shape[1]//h_states_lstm))(concat_CNN_ouputs)
    
    
#     lstm1 = layers.LSTM(S, return_sequences=True, name="LSTM_1")(concat_CNN_ouputs)
#     lstm2 = layers.LSTM(S, return_sequences=True, dropout=dropout_rate, name="LSTM_2")(lstm1)
#     final_output = layers.Dense(K+1, activation="softmax", name="FC_softmax")(lstm2[:,:,-1])

#     cascade_model = models.Model(inputs=input_layers, outputs=[final_output])
    
#     return cascade_model


# cascade_model = create_model_cascade()
# cascade_model.compile(loss=lossfn, optimizer=opt, metrics=metric)

# cascade_model.summary()

In [8]:
S = 48    # S = number of images
K = 7    # K = number of classes
h_states_lstm = 64
dropout_rate = 0.5
learning_rate = 1e-4
opt = "adam"
lossfn = "categorical_crossentropy"
metric = ["acurracy"]

In [9]:
cascade_model = models.Sequential()
cascade_model.add(layers.Input(shape=(S, 9, 9, 1)))
cascade_model.add(layers.TimeDistributed(layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu')))
cascade_model.add(layers.TimeDistributed(layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu')))
cascade_model.add(layers.TimeDistributed(layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu')))
cascade_model.add(layers.TimeDistributed(layers.Flatten()))
cascade_model.add(layers.TimeDistributed(layers.Dense(1024, activation='relu')))
cascade_model.add(layers.TimeDistributed(layers.Dropout(0.5)))
cascade_model.add(layers.LSTM(S, return_sequences=True))
cascade_model.add(layers.LSTM(S, return_sequences=True))
# cascade_model.add(layers.Dense(64, activation='relu'))
cascade_model.add(layers.Dropout(0.5))
cascade_model.add(layers.TimeDistributed(layers.Dense(S, activation='relu')))
cascade_model.add(layers.Flatten())
cascade_model.add(layers.Dense(8))
cascade_model.add(layers.Activation('softmax'))
cascade_model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(1e-4), metrics=['accuracy'])

In [10]:
cascade_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 48, 9, 9, 32)      320       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 48, 9, 9, 64)      18496     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 48, 9, 9, 128)     73856     
_________________________________________________________________
time_distributed_3 (TimeDist (None, 48, 10368)         0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 48, 1024)          10617856  
_________________________________________________________________
time_distributed_5 (TimeDist (None, 48, 1024)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 48, 48)            2

In [11]:
from numpy.lib.stride_tricks import as_strided

def sliding_window(a, w = 4, o = 2, copy = False):
    sh = (a.size - w + 1, w)
    st = a.strides * 2
    view = np.lib.stride_tricks.as_strided(a, strides = st, shape = sh)[0::o]
    if copy:
        return view.copy()
    else:
        return view

In [12]:
num_epochs = 100

a = np.arange(769)
b = sliding_window(a, S, S)
c = b.flatten()

b.shape

(16, 48)

In [13]:
batch_gen = generate_batch(ME_db, seq_perms, 4, False, [1,7])

In [48]:
output_seq, labels = next(batch_gen)

In [32]:


inputs = np.zeros((len(output_seq), len(c), 9, 9))
targets = np.zeros((len(output_seq), len(c)//S))
for i in range(len(output_seq)):
    s = output_seq[i][c]
    s = s/(np.max(s) - np.min(s))
    inputs[i] = s
    targets[i,:] = labels[i]


# targets.shape
inputs = inputs.reshape((8*b.shape[0], S, 9, 9))
inputs = np.expand_dims(inputs, axis=-1)
# targets = to_categorical(targets, dtype='uint8')
cascade_model.fit(inputs, to_categorical(targets.flatten()), batch_size=1, epochs=10)

Epoch 1/10
128/128 [==============================] - 1s 10ms/step - loss: 1.0972 - accuracy: 0.4688
Epoch 2/10
128/128 [==============================] - 1s 10ms/step - loss: 0.7049 - accuracy: 0.5078
Epoch 3/10
128/128 [==============================] - 1s 10ms/step - loss: 0.7143 - accuracy: 0.4922
Epoch 4/10
128/128 [==============================] - 1s 10ms/step - loss: 0.7131 - accuracy: 0.4531
Epoch 5/10
128/128 [==============================] - 1s 10ms/step - loss: 0.7105 - accuracy: 0.4766
Epoch 6/10
128/128 [==============================] - 1s 10ms/step - loss: 0.7098 - accuracy: 0.5000
Epoch 7/10
128/128 [==============================] - 1s 10ms/step - loss: 0.7077 - accuracy: 0.4531
Epoch 8/10
128/128 [==============================] - 1s 10ms/step - loss: 0.7197 - accuracy: 0.4766
Epoch 9/10
128/128 [==============================] - 1s 10ms/step - loss: 0.6942 - accuracy: 0.5391
Epoch 10/10
128/128 [==============================] - 1s 10ms/step - loss: 0.7048 - accura

In [64]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

print(labels)

ntrial = 4
frame = 1
diff = np.max(output_seq[ntrial][frame]) - np.min(output_seq[ntrial][frame])
print(diff)

fig1 = plt.figure(1)
fig2 = plt.figure(2)
fig1.imshow(output_seq[ntrial][frame])
fig2.plot(output_seq[ntrial, :, :].reshape((769, 81)))

[1 1 1 1 7 7 7 7]
2346.1571828025153


AttributeError: 'Figure' object has no attribute 'imshow'

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

In [17]:


# inputs = np.zeros((len(output_seq), len(c), 9, 9))
# targets = np.zeros((len(output_seq), len(c)//S))

# t1 = time.time()

# for epoch in range(num_epochs):
#     print("EPOCH %d" %(epoch))
#     batch_gen = generate_batch(ME_db, seq_perms, 4, True, [1,7])
    
#     for output_seq, labels in batch_gen:
#         inputs = np.zeros((len(output_seq), len(c), 9, 9))
#         targets = np.zeros((len(output_seq), len(c)//S))
#         for i in range(len(output_seq)):
#             inputs[i] = output_seq[i][c]/np.max(output_seq[i][c])
#             targets[i,:] = labels[i]


#         # targets.shape
#         inputs = inputs.reshape((8*b.shape[0], S, 9, 9))
#         inputs = np.expand_dims(inputs, axis=-1)
#         # targets = to_categorical(targets, dtype='uint8')
#         cascade_model.fit(inputs, to_categorical(targets.flatten()), batch_size=inputs.shape[0]//8, epochs=10)
        
# print('Time in total:', time.time()-t1)


In [18]:
inputs1 = layers.Input(shape=(2, 3))
lstm1, state_h, state_c = layers.LSTM(2, return_sequences=True, return_state=True)(inputs1)
model = models.Model(inputs=inputs1, outputs=[state_h])
# define input data
data = np.array([[0.1, 0.2, 0.3], [0.1, 0.2, 0.3]]).reshape((1,2,3))
# make and show prediction
print(model.predict(data))
model

[[ 0.06434825 -0.07697313]]
